In [5]:
from sklearn.ensemble import RandomForestClassifier
from etl import UserGenerator
from feature_engineer import FeatureEngineer
from train import Train

# 1) Cargar datos
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00267/data_banknote_authentication.txt"
user_generator = UserGenerator(url)
df = user_generator.create_dataset()

# 2) Ingeniería de características
feature_engineer = FeatureEngineer(df)
df_engineered = feature_engineer.create_features()

# 3) Configuración de features y target
numeric_features = [
    "variance", "skewness", "curtosis", "entropy",
    "var_entropy_ratio", "magnitude", "abs_skewness", "curtosis_minus_skewness"
]
categorical_features = ["bucket_curtosis"]
target_column = "class"
test_size = 0.25

# 4) Modelo
model = RandomForestClassifier(random_state=42, n_estimators=100)

# 5) Entrenamiento
train = Train(
    df=df_engineered,
    numeric_features=numeric_features,
    categorical_features=categorical_features,   # 👈 ESTA FALTABA
    target_column=target_column,
    model=model,
    test_size=test_size
)

pipeline, X_test, y_test = train.train()

# 6) Mostrar pipeline entrenado
pipeline


📥 Cargando dataset de billetes...

# Nulos por columna
variance    0
skewness    0
curtosis    0
entropy     0
class       0
dtype: int64

# Distribución de la clase (0=auténtico, 1=falso)
class
0    762
1    610
Name: count, dtype: int64

Proporción:
class
0    0.555
1    0.445
Name: proportion, dtype: float64


2025/09/21 13:25:01 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '1998c6e9cc2640d29397fcbffd2f87e7', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow


🏃 View run wise-grub-66 at: http://127.0.0.1:5000/#/experiments/537002863342184639/runs/1998c6e9cc2640d29397fcbffd2f87e7
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/537002863342184639


,steps,"[('preprocessor', ...), ('classifier', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [6]:
df_engineered

,variance,skewness,curtosis,entropy,class,var_entropy_ratio,magnitude,bucket_curtosis,abs_skewness,curtosis_minus_skewness
0,3.62160,8.66610,-2.8073,-0.44699,0,-8.102195,9.813155,low,8.66610,-11.47340
1,4.54590,8.16740,-2.4586,-1.46210,0,-3.109158,9.775177,low,8.16740,-10.62600
2,3.86600,-2.63830,1.9242,0.10645,0,36.317520,5.061666,medium,2.63830,4.56250
3,3.45660,9.52280,-4.0112,-3.59440,0,-0.961663,11.473502,low,9.52280,-13.53400
4,0.32924,-4.45520,4.5718,-0.98880,0,-0.332969,6.468098,high,4.45520,9.02700
...,...,...,...,...,...,...,...,...,...,...
1367,0.40614,1.34920,-1.4501,-0.55949,1,-0.725911,2.097882,low,1.34920,-2.79930
1368,-1.38870,-4.87730,6.4774,0.34179,1,-4.063021,8.233473,high,4.87730,11.35470
1369,-3.75030,-13.45860,17.5932,-2.77710,1,1.350438,22.636953,high,13.45860,31.05180
1370,-3.56370,-8.38270,12.3930,-1.28230,1,2.779147,15.433741,high,8.38270,20.77570


In [7]:
import sys
print(sys.executable)


c:\Users\Usuario\Phyton 1\venv\Scripts\python.exe


In [8]:
import mlflow
print("MLflow version:", mlflow.__version__)


MLflow version: 3.4.0


In [9]:
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestClassifier
from train_with_mlflow import TrainMlflow  # 👈 Importar clase

# ==========================
# Configuración de MLflow
# ==========================
experiment_name = "billetes-random-forest"
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment(experiment_name)

mlflow.sklearn.autolog(
    log_input_examples=True,
    log_model_signatures=True,
    log_models=True,
    disable=False,
    exclusive=False,
    disable_for_unsupported_versions=False,
    silent=False,
    max_tuning_runs=5
)

# ==========================
# Features y target
# ==========================
numeric_features = [
    "variance", "skewness", "curtosis", "entropy",
    "var_entropy_ratio", "magnitude", 
    "abs_skewness", "curtosis_minus_skewness"
]
target_column = "class"
test_size = 0.25

# ==========================
# Experimento con Random Forest
# ==========================
params_random_forest = {
    'n_estimators': 100,
    'max_depth': 10,
    'min_samples_split': 2,
    'min_samples_leaf': 1
}
model = RandomForestClassifier(random_state=42, **params_random_forest)

train_mlflow = TrainMlflow(
    df=df_engineered,
    numeric_features=numeric_features,
    target_column=target_column,
    model=model,
    test_size=test_size,
    model_params=params_random_forest,
    mlflow_setup=mlflow
)

pipeline, run_id = train_mlflow.train()

print("\n🏃 View run at:", f"http://127.0.0.1:5000/#/experiments/1/runs/{run_id}")


MLflow Run ID: 7f67e48174514bfaad7c445a249c959d
Tracking URI: http://127.0.0.1:5000
Train Accuracy: 1.0000
Test Accuracy: 0.9942
🏃 View run magnificent-bee-712 at: http://127.0.0.1:5000/#/experiments/537002863342184639/runs/7f67e48174514bfaad7c445a249c959d
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/537002863342184639

🏃 View run at: http://127.0.0.1:5000/#/experiments/1/runs/7f67e48174514bfaad7c445a249c959d


In [2]:
import xgboost
print("XGBoost version:", xgboost.__version__)


XGBoost version: 3.0.5


In [11]:
from xgboost import XGBClassifier
import mlflow
import mlflow.xgboost

# ==========================
# Configuración de MLflow
# ==========================
experiment_name = "billetes-xgboost"
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment(experiment_name)

# Habilitar autologging para XGBoost
mlflow.xgboost.autolog()

# ==========================
# Features y target
# ==========================
numeric_features = [
    "variance", "skewness", "curtosis", "entropy",
    "var_entropy_ratio", "magnitude", 
    "abs_skewness", "curtosis_minus_skewness"
]
# ⚡️ Ya no pasamos categorical_features porque tu clase TrainMlflow no lo recibe
target_column = "class"
test_size = 0.25

# ==========================
# Experimento con XGBoost
# ==========================
params_xgb = {
    'n_estimators': 100,
    'max_depth': 10,
    'learning_rate': 0.1,    # reemplazo de min_samples_split/leaf
    'subsample': 0.8,        # control de muestreo
    'colsample_bytree': 0.8  # control de features
}

model_xgb = XGBClassifier(
    random_state=42,
    **params_xgb
)

# ==========================
# Entrenamiento con TrainMlflow
# ==========================
train_mlflow = TrainMlflow(
    df=df_engineered,
    numeric_features=numeric_features,
    target_column=target_column,
    model=model_xgb,
    test_size=test_size,
    model_params=params_xgb,
    mlflow_setup=mlflow
)

pipeline, run_id = train_mlflow.train()

print("\n🏃 View run at:", f"http://127.0.0.1:5000/#/experiments/2/runs/{run_id}")


MLflow Run ID: d9557ff30b02491398b6069c26223e37
Tracking URI: http://127.0.0.1:5000
Train Accuracy: 1.0000
Test Accuracy: 1.0000
🏃 View run gifted-hare-354 at: http://127.0.0.1:5000/#/experiments/321371579144275152/runs/d9557ff30b02491398b6069c26223e37
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/321371579144275152

🏃 View run at: http://127.0.0.1:5000/#/experiments/2/runs/d9557ff30b02491398b6069c26223e37


In [12]:
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestClassifier
from train_with_mlflow import TrainMlflow  # 👈 Importar clase

# ==========================
# Configuración de MLflow
# ==========================
experiment_name = "billetes-random-forest-V2"   # 👈 nuevo experimento
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment(experiment_name)

mlflow.sklearn.autolog(
    log_input_examples=True,
    log_model_signatures=True,
    log_models=True,
    disable=False,
    exclusive=False,
    disable_for_unsupported_versions=False,
    silent=False,
    max_tuning_runs=5
)

# ==========================
# Features y target
# ==========================
numeric_features = [
    "variance", "skewness", "curtosis", "entropy",
    "var_entropy_ratio", "magnitude", 
    "abs_skewness", "curtosis_minus_skewness"
]
target_column = "class"
test_size = 0.25

# ==========================
# Experimento con Random Forest (V2 mejorado)
# ==========================
params_random_forest_v2 = {
    'n_estimators': 1000,   # más árboles
    'max_depth': None,      # sin límite
    'min_samples_split': 2,
    'min_samples_leaf': 1,
    'max_features': None,   # usar todas las variables
    'bootstrap': True
}

model_v2 = RandomForestClassifier(random_state=42, **params_random_forest_v2)

train_mlflow_v2 = TrainMlflow(
    df=df_engineered,
    numeric_features=numeric_features,
    target_column=target_column,
    model=model_v2,
    test_size=test_size,
    model_params=params_random_forest_v2,
    mlflow_setup=mlflow
)

pipeline_v2, run_id_v2 = train_mlflow_v2.train()

print("\n🏃 View run V2 at:", f"http://127.0.0.1:5000/#/experiments/2/runs/{run_id_v2}")


2025/09/21 13:34:45 INFO mlflow.tracking.fluent: Experiment with name 'billetes-random-forest-V2' does not exist. Creating a new experiment.


MLflow Run ID: ca3b67dc438e4b479c6108150671890c
Tracking URI: http://127.0.0.1:5000
Train Accuracy: 1.0000
Test Accuracy: 0.9913
🏃 View run big-lynx-21 at: http://127.0.0.1:5000/#/experiments/511832567646551150/runs/ca3b67dc438e4b479c6108150671890c
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/511832567646551150

🏃 View run V2 at: http://127.0.0.1:5000/#/experiments/2/runs/ca3b67dc438e4b479c6108150671890c


In [13]:
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestClassifier
from train_with_mlflow_optuna import TrainMlflowOptuna  # 👈 tu clase con Optuna


# ==========================
# Configuración de MLflow
# ==========================
experiment_name = "billetes-random-forest-optuna"
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment(experiment_name)


# ==========================
# Features y target
# ==========================
numeric_features = [
    "variance", "skewness", "curtosis", "entropy",
    "var_entropy_ratio", "magnitude", 
    "abs_skewness", "curtosis_minus_skewness"
]
categorical_features = ["bucket_curtosis"]   # generado en tu FeatureEngineer
target_column = "class"


# ==========================
# Espacio de búsqueda Optuna
# ==========================
param_distributions = {
    'n_estimators': ('int', 50, 200),
    'max_depth': ('int', 5, 30),
    'min_samples_split': ('int', 2, 10),
    'min_samples_leaf': ('int', 1, 5),
    'max_features': ('categorical', ['sqrt', 'log2', None])
}


# ==========================
# Entrenador con Optuna
# ==========================
trainer = TrainMlflowOptuna(
    df=df_engineered,
    numeric_features=numeric_features,
    categorical_features=categorical_features,
    target_column=target_column,
    model_class=RandomForestClassifier,
    test_size=0.3,
    n_trials=30,                      # número de intentos de Optuna
    optimization_metric='f1',         # métrica a optimizar
    param_distributions=param_distributions,
    model_params={'random_state': 42, 'n_jobs': -1},
    mlflow_setup=mlflow
)


# ==========================
# Ejecutar optimización
# ==========================
best_pipeline, run_id, study = trainer.train()

print("\n🏃 View best run at:", f"http://127.0.0.1:5000/#/experiments/4/runs/{run_id}")
print("📊 Best parameters:", study.best_params)
print("⭐ Best F1 Score:", study.best_value)


2025/09/21 13:44:01 INFO mlflow.tracking.fluent: Experiment with name 'billetes-random-forest-optuna' does not exist. Creating a new experiment.
[I 2025-09-21 13:44:02,040] A new study created in memory with name: optuna_RandomForestClassifier
2025/09/21 13:44:02 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'f5d088799af0433b93b45fc10d3d7c47', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow


[I 2025-09-21 13:44:08,162] Trial 0 finished with value: 0.9975721346155462 and parameters: {'n_estimators': 187, 'max_depth': 5, 'min_samples_split': 9, 'min_samples_leaf': 5, 'max_features': 'sqrt'}. Best is trial 0 with value: 0.9975721346155462.


🏃 View run dashing-loon-443 at: http://127.0.0.1:5000/#/experiments/519298823862010791/runs/f5d088799af0433b93b45fc10d3d7c47
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/519298823862010791


2025/09/21 13:44:08 INFO mlflow.tracking.fluent: Experiment with name 'optuna_RandomForestClassifier' does not exist. Creating a new experiment.
2025/09/21 13:44:08 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '36e139386f614c16960941d5d20dd2e9', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow


🏃 View run 0 at: http://127.0.0.1:5000/#/experiments/520885596519322455/runs/07918ec932914a54a6d66d8283c52275
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/520885596519322455


...

🏃 View run 28 at: http://127.0.0.1:5000/#/experiments/520885596519322455/runs/aa5e85894bcb491bbccd41817b6a8415
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/520885596519322455


[I 2025-09-21 13:48:02,317] Trial 29 finished with value: 0.9951428478655157 and parameters: {'n_estimators': 136, 'max_depth': 21, 'min_samples_split': 3, 'min_samples_leaf': 5, 'max_features': 'log2'}. Best is trial 0 with value: 0.9975721346155462.


🏃 View run honorable-squid-521 at: http://127.0.0.1:5000/#/experiments/520885596519322455/runs/aab4d0bff357459983d1b91815a258b3
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/520885596519322455
🏃 View run 29 at: http://127.0.0.1:5000/#/experiments/520885596519322455/runs/200abe7c108c4495b1f422caf5d981b5
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/520885596519322455

Optimization complete!
Best f1: 0.9976
Best parameters: {'n_estimators': 187, 'max_depth': 5, 'min_samples_split': 9, 'min_samples_leaf': 5, 'max_features': 'sqrt'}


2025/09/21 13:48:09 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/21 13:48:12 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



Best Model MLflow Run ID: ddce2ebc1bfd49219990ee98f234eb56
Tracking URI: http://127.0.0.1:5000
🏃 View run best_model_RandomForestClassifier at: http://127.0.0.1:5000/#/experiments/520885596519322455/runs/ddce2ebc1bfd49219990ee98f234eb56
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/520885596519322455

🏃 View best run at: http://127.0.0.1:5000/#/experiments/4/runs/ddce2ebc1bfd49219990ee98f234eb56
📊 Best parameters: {'n_estimators': 187, 'max_depth': 5, 'min_samples_split': 9, 'min_samples_leaf': 5, 'max_features': 'sqrt'}
⭐ Best F1 Score: 0.9975721346155462


In [14]:
import optuna

# Importar visualización de Optuna
optuna.visualization.plot_optimization_history(study).show()
optuna.visualization.plot_param_importances(study).show()


2025/09/21 13:54:01 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'e8236748a0ca467aba81581e0c8f8b45', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow


🏃 View run omniscient-hen-391 at: http://127.0.0.1:5000/#/experiments/520885596519322455/runs/e8236748a0ca467aba81581e0c8f8b45
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/520885596519322455


In [15]:
best_model = trainer.best_pipeline
y_pred = best_model.predict(X_test)

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.99      1.00      1.00       191
           1       1.00      0.99      1.00       152

    accuracy                           1.00       343
   macro avg       1.00      1.00      1.00       343
weighted avg       1.00      1.00      1.00       343



In [17]:
# Nombre del modelo
print("Best model:", trainer.best_model.__class__.__name__)


Best model: RandomForestClassifier


In [18]:
print("Best parameters:", trainer.best_params)


Best parameters: {'n_estimators': 187, 'max_depth': 5, 'min_samples_split': 9, 'min_samples_leaf': 5, 'max_features': 'sqrt'}


In [19]:
print("Experiment name:", experiment_name)
print("Best run ID:", run_id)


Experiment name: billetes-random-forest-optuna
Best run ID: ddce2ebc1bfd49219990ee98f234eb56
